In [17]:
import torch
import torchvision as tv
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import time

In [2]:
training_dataset = tv.datasets.FashionMNIST(
    root="data",
    train=True,
    download=True, 
    transform=tv.transforms.ToTensor(),
    target_transform=None
)

test_dataset = tv.datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=tv.transforms.ToTensor(),
    target_transform=None
)


1.2%

100.0%


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw


100.0%


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




5.9%

100.0%


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [25]:
BATCH_SIZE = 32

train_dataloader = torch.utils.data.DataLoader(
    training_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)

train_features_batch, train_labels_batch = next(iter(train_dataloader))
train_features_batch.shape, train_labels_batch.shape

(torch.Size([32, 1, 28, 28]), torch.Size([32]))

In [6]:
#Baseline model:

class Model0(torch.nn.Module):
    def __init__(self, input_shape: int, hidden_layers: int, output_shape: int):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.Linear(input_shape, hidden_layers),
            torch.nn.Linear(hidden_layers, output_shape)
        )

    def forward(self, x):
        return self.layers(x)

In [7]:
model0 = Model0(28*28, 10, 10)
model0

Model0(
  (layers): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
  )
)

In [24]:
dummy_x = torch.randn(1, 28, 28)
print(model0(dummy_x).shape)

for X_test, y_test in test_dataloader:

            test_pred = model0(X_test)
print(test_pred.shape)

torch.Size([1, 10])
torch.Size([16, 10])


In [14]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    accuracy = correct / len(y_true) * 100
    return accuracy

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model0.parameters(), lr=0.01)

def print_train_time(start: float, end: float):
    total_time = end - start
    print("Training time: ", total_time, "seconds")
    return total_time


In [29]:
#Pętla

#for epochs:
    #for batches:
        #training...

train_time_start = time.time()

EPOCHS = 5

for epoch in tqdm(range(EPOCHS)):
    print("Epoch: ", epoch, "\n--------------------------")
    train_loss = 0
    batch = 0
    for batch, (X, y) in enumerate(train_dataloader):
        batch += 1
        model0.train()
        y_pred = model0(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            print("Epoch ", epoch, " Batch: ", batch)
    
    train_loss /= len(train_dataloader)

    #test

    test_acc, test_loss = 0, 0

    model0.eval()
    with torch.inference_mode():
        for X_test, y_test in test_dataloader:

            y_pred = model0(X_test)
            test_loss += loss_fn(y_pred, y_test)
            #test_acc += accuracy_fn(y_test, y_pred)
        test_loss /= len(test_dataloader)
        test_acc /= len(test_dataloader)
        
        print("Epoch: ", epoch, " | Train loss: ", train_loss.item(), " | Test loss: ", test_loss.item(), " | Test accuracy: ", test_acc)

train_time_end = time.time()
total_train_time = print_train_time(train_time_start, train_time_end)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch:  0 
--------------------------
Epoch  0  Batch:  100
Epoch  0  Batch:  200
Epoch  0  Batch:  300
Epoch  0  Batch:  400
Epoch  0  Batch:  500
Epoch  0  Batch:  600
Epoch  0  Batch:  700
Epoch  0  Batch:  800
Epoch  0  Batch:  900
Epoch  0  Batch:  1000
Epoch  0  Batch:  1100
Epoch  0  Batch:  1200
Epoch  0  Batch:  1300
Epoch  0  Batch:  1400
Epoch  0  Batch:  1500
Epoch  0  Batch:  1600
Epoch  0  Batch:  1700
Epoch  0  Batch:  1800


 20%|██        | 1/5 [00:10<00:40, 10.05s/it]

Epoch:  0  | Train loss:  0.42680442333221436  | Test loss:  0.47794559597969055  | Test accuracy:  0.0
Epoch:  1 
--------------------------
Epoch  1  Batch:  100
Epoch  1  Batch:  200
Epoch  1  Batch:  300
Epoch  1  Batch:  400
Epoch  1  Batch:  500
Epoch  1  Batch:  600
Epoch  1  Batch:  700
Epoch  1  Batch:  800
Epoch  1  Batch:  900
Epoch  1  Batch:  1000
Epoch  1  Batch:  1100
Epoch  1  Batch:  1200
Epoch  1  Batch:  1300
Epoch  1  Batch:  1400
Epoch  1  Batch:  1500
Epoch  1  Batch:  1600
Epoch  1  Batch:  1700
Epoch  1  Batch:  1800


 40%|████      | 2/5 [00:19<00:29,  9.73s/it]

...

Epoch:  2  | Train loss:  0.4188637137413025  | Test loss:  0.4551538825035095  | Test accuracy:  0.0
Epoch:  3 
--------------------------
Epoch  3  Batch:  100
Epoch  3  Batch:  200
Epoch  3  Batch:  300
Epoch  3  Batch:  400
Epoch  3  Batch:  500
Epoch  3  Batch:  600
Epoch  3  Batch:  700
Epoch  3  Batch:  800
Epoch  3  Batch:  900
Epoch  3  Batch:  1000
Epoch  3  Batch:  1100
Epoch  3  Batch:  1200
Epoch  3  Batch:  1300
Epoch  3  Batch:  1400
Epoch  3  Batch:  1500
Epoch  3  Batch:  1600
Epoch  3  Batch:  1700
Epoch  3  Batch:  1800


 80%|████████  | 4/5 [00:38<00:09,  9.70s/it]

Epoch:  3  | Train loss:  0.41496342420578003  | Test loss:  0.4595507085323334  | Test accuracy:  0.0
Epoch:  4 
--------------------------
Epoch  4  Batch:  100
Epoch  4  Batch:  200
Epoch  4  Batch:  300
Epoch  4  Batch:  400
Epoch  4  Batch:  500
Epoch  4  Batch:  600
Epoch  4  Batch:  700
Epoch  4  Batch:  800
Epoch  4  Batch:  900
Epoch  4  Batch:  1000
Epoch  4  Batch:  1100
Epoch  4  Batch:  1200
Epoch  4  Batch:  1300
Epoch  4  Batch:  1400
Epoch  4  Batch:  1500
Epoch  4  Batch:  1600
Epoch  4  Batch:  1700
Epoch  4  Batch:  1800


100%|██████████| 5/5 [00:48<00:00,  9.78s/it]

Epoch:  4  | Train loss:  0.4125295877456665  | Test loss:  0.4488404095172882  | Test accuracy:  0.0
Training time:  48.91603875160217 seconds


In [ ]:
def eval_model(model: torch.nn.Module, data_loader = torch.utils.data.DataLoader, loss_fn: torch.nn.Module, accuracy_fn):

    
